<a href="https://colab.research.google.com/github/data-aleks/billboard_hot100_tidytuesday_26_08_2025/blob/main/tidy_tuesday_26082025_billboard_hot_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

# **Billboard Hot 100 Number Ones - Tidy Tuesday - 26.08.2025**

This week we are exploring the Billboard Hot 100 Number Ones Database. This workbook contains substantial data about every song to ever top the Billboard Hot 100 between August 4, 1958 and January 11, 2025. It was compiled by Chris Dalla Riva as he wrote the book Uncharted Territory: What Numbers Tell Us about the Biggest Hit Songs and Ourselves. It also often powers his newsletter Can't Get Much Higher.

7 years ago, I decided that I was going to listen to every number one hit. Along the way, I tracked an absurd amount of information about each song. Using that information, I wrote a data-driven history of popular music covering 1958 through today.

- Have #1 hits become shorter over time?
- Does the relation between artist age and chart success change across time?
- Which keys are most common in #1 hits? Do our key preferences differ by genre?
- What lyrical topics have dominated #1 hits across different decades?
- How has the prevalence of explicit content changed over time?

Thank you to Jen Richmond (R-Ladies Sydney) for curating this week's dataset.

In [10]:
df_billboard = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-08-26/billboard.csv')
df_topics = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-08-26/topics.csv')


In [11]:
if df_billboard.empty or df_topics.empty:
  print('Couldnt get the data')
else:
  print('Data loaded succesfully')

Data loaded succesfully


## **Step 1.** Exploring dataset

In this step we will look at our dataframe, see if there is missing values and decide how we will approach cleaning the dataset to be used in future steps.

In [12]:
df_billboard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1177 entries, 0 to 1176
Columns: 105 entries, song to u_s_artwork
dtypes: float64(17), int64(61), object(27)
memory usage: 965.6+ KB


In [13]:
df_topics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   lyrical_topics  97 non-null     object
dtypes: object(1)
memory usage: 908.0+ bytes


This is very wide dataset, the best way to check for missing values would be to show us the columns that have more than 0 missing values.

In [14]:
missing_counts = df_billboard.isnull().sum()
missing_counts[missing_counts > 0].sort_values(ascending=False)

,0
featured_in_a_then_contemporary_play,1175
talent_contestant,1167
double_a_side,1165
featured_in_a_then_contemporary_t_v_show,1152
sound_effects,1113
featured_in_a_then_contemporary_film,1073
featured_artists,988
discogs_style,94
cdr_style,88
cdr_genre,88


As we can see here, this is a very wide dataset with some columns missing quite a few of the values. At this stage we need to decide how to approach this? Is working with 100+ columns optimal not only from analysis perspective but also from performance perspective. We can't even view the entire 100+ columns using standard .info() , i think the best way to work with this dataset is to only use what we need to answer our questions. This will benefit overall technical perforamnce, but will also make working with the dataset easier.

# **Step 2.** Cleaning dataset

This is a very wide datset, with over 100 columns of data. While i would love to utilise every single column of data and use the entirety of this dataset i think the best approach would be to extract selected columns in to a new dataframe to make working with this data easier. We could always add extra columns if neccesary. This will prevent us from carrying out unnecessary data cleaning steps, again if there is need for any extra columns that have missing values we could always add these if needed. Initially i will extract the data required to deal with the initial questions.

### **Step 2.1** Extracting data required for our analysis

* 'Have #1 hits become shorter over time' for this question we will need date the record is released and it's length in seconds.
* 'Does the relation between artist age and chart success change across time?' we will need front_person_age
* 'Which keys are most common in #1 hits? Do our key preferences differ by genre?' we will need song key information, luckily we have both key and simplified_key information, i think simplified key will serve us best as we need to know general key of the record.
* 'What lyrical topics have dominated #1 hits across different decades?' this is simple again, we will only require lyrical_topic column
* 'How has the prevalence of explicit content changed over time?' we have a column which identifies records that have explicit content.

In [15]:
df_billboard_short = df_billboard[['date', 'artist', 'front_person_age', 'song', 'length_sec', 'keys', 'lyrical_topic',  'explicit']]

In [16]:
df_billboard_short.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1177 entries, 0 to 1176
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              1177 non-null   object 
 1   artist            1177 non-null   object 
 2   front_person_age  1155 non-null   float64
 3   song              1177 non-null   object 
 4   length_sec        1177 non-null   int64  
 5   keys              1177 non-null   object 
 6   lyrical_topic     1151 non-null   object 
 7   explicit          1177 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 73.7+ KB


In [17]:
df_billboard_short.head()

,date,artist,front_person_age,song,length_sec,keys,lyrical_topic,explicit
0,1958-08-04T00:00:00Z,Ricky Nelson,18.0,Poor Little Fool,154,C,Lost Love,0
1,1958-08-18T00:00:00Z,Domenico Modugno,30.0,Nel Blu Dipinto Di Blu,219,Bb,Flying;Dreaming,0
2,1958-08-25T00:00:00Z,The Elegants,17.0,Little Star,163,A,Longing for Love,0
3,1958-09-29T00:00:00Z,Tommy Edwards,35.0,It's All in the Game,156,Eb,Love,0
4,1958-11-10T00:00:00Z,Conway Twitty,25.0,It's Only Make Believe,134,B,Lost Love,0


Now we have a very condensed dataset containing only data we need to answer the questions we need to answer.

### **Step 2.2** Dealing with missing values in the dataset.

The dataset still has some missing values, which i do not like. The total number of rows with missing data is approximately 48 which would represent around 4.1 % of our data. There is a couple of way we are able to approach this. We could potentially attempt to fill the missing data but give the low number of rows affected i think the most simplest approach would be to remove rows with missing values as it's below 5% of our entire data and would not affect our analysis.

In [18]:
df_billboard_short[df_billboard_short['front_person_age'].isnull()]

,date,artist,front_person_age,song,length_sec,keys,lyrical_topic,explicit
47,1961-04-03T00:00:00Z,The Marcels,NaN,Blue Moon,135,G,Longing for Love;Love,0
62,1961-12-18T00:00:00Z,The Tokens,NaN,The Lion Sleeps Tonight,162,F,Unclear,0
81,1962-12-22T00:00:00Z,The Tornados,NaN,Telstar,195,A;D,NaN,0
94,1963-08-03T00:00:00Z,The Tymes,NaN,So Much in Love,140,E,Love,0
192,1967-11-25T00:00:00Z,Strawberry Alarm Clock,NaN,Incense and Peppermints,167,G,Utopia/Perfect Places;Love;Coexistence,0
218,1969-09-20T00:00:00Z,The Archies,NaN,"Sugar, Sugar",168,D,Love,0
253,1971-06-12T00:00:00Z,Honey Cone,NaN,Want Ads,166,Ab,Longing for Love;Bad Relationships,0
323,1974-04-20T00:00:00Z,MFSB,NaN,TSOP (The Sound of Philadelphia),212,C,NaN,0
327,1974-06-15T00:00:00Z,Bo Donaldson & the Heywoods,NaN,Billy Don't Be a Hero,207,Bb;B&%,Love;Death;War,0
352,1975-02-08T00:00:00Z,Ohio Players,NaN,Fire,192,Cm;Dbm&%,Lust/Sex,0


In [19]:
df_billboard_short[df_billboard_short['lyrical_topic'].isnull()]

,date,artist,front_person_age,song,length_sec,keys,lyrical_topic,explicit
12,1959-05-11T00:00:00Z,"Dave ""Baby"" Cortez",20.0,The Happy Organ,122,Db,NaN,0
18,1959-09-21T00:00:00Z,Santo & Johnny,19.5,Sleep Walk,151,C,NaN,0
26,1960-02-22T00:00:00Z,Percy Faith & His Orchestra,51.0,Theme from A Summer Place,144,F;D%,NaN,0
42,1961-01-09T00:00:00Z,Bert Kaempfert,37.0,Wonderland by Night,194,Eb;E&%,NaN,0
44,1961-02-13T00:00:00Z,Lawrence Welk,57.0,Calcutta,136,G,NaN,0
70,1962-05-26T00:00:00Z,Mr. Acker Bilk,33.0,Stranger on the Shore,172,Eb,NaN,0
72,1962-07-07T00:00:00Z,David Rose & His Orchestra,52.0,The Stripper,116,F,NaN,0
81,1962-12-22T00:00:00Z,The Tornados,NaN,Telstar,195,A;D,NaN,0
95,1963-08-10T00:00:00Z,Stevie Wonder,13.0,Fingertips Pt. 2,190,Cm,NaN,0
197,1968-02-10T00:00:00Z,Paul Mauriat,42.0,Love is Blue,151,Em;E,NaN,0


In [20]:
df_billboard_short = df_billboard_short.dropna()

In [21]:
df_billboard_short.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1132 entries, 0 to 1176
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              1132 non-null   object 
 1   artist            1132 non-null   object 
 2   front_person_age  1132 non-null   float64
 3   song              1132 non-null   object 
 4   length_sec        1132 non-null   int64  
 5   keys              1132 non-null   object 
 6   lyrical_topic     1132 non-null   object 
 7   explicit          1132 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 79.6+ KB
